In [102]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from random import seed
from sklearn import preprocessing
seed(1)

In [124]:
class conv_lstm(nn.Module):
    def __init__(self, lr, input_channels = 1, batch_size = 1, lstm_hidden_size = 512, lstm_layer_size = 2):
        super(conv_lstm, self).__init__()
        self.lr = lr
        self.input_channels = input_channels
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #self.height = height
        #self.width = width
        self.batch_size = batch_size
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm_layer_size = lstm_layer_size
        self.conv_layer = nn.Conv2d(in_channels = self.input_channels, out_channels = 10,
                                   kernel_size = 3, stride = 1)
        conv_layer_init_ = 1.0/np.sqrt(self.conv_layer.weight.data.size()[0])
        self.conv_layer.weight.data.uniform_(-conv_layer_init_, conv_layer_init_)
        self.conv_layer.bias.data.uniform_(-conv_layer_init_, conv_layer_init_)
        self.conv_layer_batchNorm = nn.BatchNorm2d(10)
        self.pool = nn.MaxPool2d((2,4), stride=1)#experiment with maxpool/avgpool
        #second querry - should i maxpool/avgpool then send to lstm layer or leave it as it is? Refer to paper
        
        #entirely dependant on convlayer out shape
        self.lstm_layer = nn.LSTM(input_size = 5,#check forward shape and last shape: adjust accordingly
                                 hidden_size = self.lstm_hidden_size,
                                 num_layers  = self.lstm_layer_size,
                                 batch_first = True,
                                 dropout = 0.1)
        
        self.fc_layer = nn.Linear(self.lstm_hidden_size, 10)#10 to represebt feature length
        fc_init_ = 1.0/np.sqrt(self.fc_layer.weight.data.size()[0])
        #initialize weights and biases
        self.fc_layer.weight.data.uniform_(-fc_init_, fc_init_)
        self.fc_layer.bias.data.uniform_(-fc_init_, fc_init_)
        
        self.to(self.device)
        
    def forward(self, frames):
        x = self.conv_layer(frames)
        x = self.conv_layer_batchNorm(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.squeeze()
        batch_size = x.size(0)
        hidden_state = torch.zeros(self.lstm_layer_size, 
                                  batch_size,
                                  self.lstm_hidden_size).to(self.device)
        cell_state = torch.zeros(self.lstm_layer_size,
                                batch_size,
                                self.lstm_hidden_size).to(self.device)
        hidden_lstm_layer = (hidden_state, cell_state)
        out, (hn,cn) = self.lstm_layer(x,(hidden_state, cell_state))
        
        #print("lstm_out shape: ", out.shape)
        modified_out = out[:, -1, :]
        fc_out = self.fc_layer(modified_out)
        return fc_out, out, hn

In [125]:
model = conv_lstm(lr = 0.001)

In [126]:
model

conv_lstm(
  (conv_layer): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer_batchNorm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=(2, 4), stride=1, padding=0, dilation=1, ceil_mode=False)
  (lstm_layer): LSTM(5, 512, num_layers=2, batch_first=True, dropout=0.1)
  (fc_layer): Linear(in_features=512, out_features=10, bias=True)
)

In [76]:
tensor_ = torch.rand(10, 10)

In [80]:
tensor_.shape#B,C,H,W

torch.Size([1, 1, 10, 10])

In [79]:
tensor_ = tensor_.unsqueeze(dim = 0)

In [129]:
out_, _,__= model(tensor_.to("cpu"))
out_.shape

torch.Size([10, 10])

In [128]:
#Implement dataloader
from torch.utils.data import DataLoader

In [ ]:
#how do i wanna load data?
#input: frame one
#output(label): frame 2